In [ ]:
import os
import re
from time import sleep

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

In [ ]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get('https://quillbot.com/summarize')

In [ ]:
FOLDER_PATH: str = 'C:\\Users\\Sharvin\\Downloads\\_Unsorted\\__To_Check\\DanMachi - Volume 12 [Yen Press][Kobo] (1)'

In [ ]:
def make_chunks(text: str) -> list[list[str]]:
    """Split the text into 600 words chunks."""
    counter: int = 0
    chunks: list[list[str]] = []
    sentences: list[str] = []
    for sentence in text.split('\n'):
        if len(sentence.split()) + counter < 600:
            sentences.append(sentence)
            counter: int = counter + len(sentence.split())
        else:
            chunks.append(sentences[:])
            counter: int = 0
            sentences.clear()
            sentences.append(sentence)
            counter: int = counter + len(sentence.split())
    chunks.append(sentences[:])
    return chunks

In [ ]:
def get_summary(text) -> str:
    textbox = driver.find_element(By.ID, 'inputBoxSummarizer')
    textbox.clear()
    sleep(1)
    textbox.send_keys(text)
    sleep(10)
    ActionChains(driver).key_down(Keys.CONTROL).key_down(Keys.ENTER).key_up(Keys.CONTROL).key_up(Keys.ENTER).perform()
    sleep(10)
    summary = driver.find_element(By.ID, 'outputBoxSummarizer')
    return summary.text

In [ ]:
def save_to_file(text: str, file: str):
    file_part: str = 'summary' + ' - ' + file.lower()
    SUMMARY: str = os.path.join(FOLDER_PATH, 'summary')
    file_summary: str = os.path.join(SUMMARY, file_part)
    if not os.path.exists(SUMMARY):
        os.mkdir(os.path.join(SUMMARY))
    with open(file_summary, 'a') as f:
        f.write(text)
        f.write('\n\n')

In [ ]:
for file in os.listdir(FOLDER_PATH):
    file_path: str = os.path.join(FOLDER_PATH, file)
    if not os.path.isfile(file_path):
        continue
    print(file)
    with open(file_path, 'r') as f:
        text: str  = f.read()
        text: str = re.sub(r'\n{2,50}', '\n', text).strip()
        parts: list[list[str]] = make_chunks(text)
        for idx, part in enumerate(parts):
            print(f'{idx + 1}/{len(parts)}')
            text: str = '\n'.join(part)
            summary_text: str = get_summary(text)
            save_to_file(summary_text, file)
driver.quit()

In [ ]:
print('COMPLETED')